In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import pipeline
import gc
import torch

ModuleNotFoundError: No module named 'datasets'

In [7]:
# Токенизатор от базовой модели
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B", trust_remote_code=True)

# Веса
model_name_or_path = "Ponimash/Qwen2.5-nli-7b"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [8]:
kd_list = ["general", "math", "biology", "legal", "physics", "medicine", "psychology", "economy", "ML/AI", "IT"]

f_i = "['txt', 'csv', 'json', 'png', 'flac', 'ogg', 'jpg', 'docx', 'pptx', 'pdf', 'mp3', 'wav']"
s_i = "['matrix', 'vector', 'tensor', 'signal', 'text', 'image', 'sound']"
st_i = "['raw', 'specter', 'cepstrum', 'speak', 'question']"
st_o = "['specter', 'cepstrum', 'speak', 'answer', 'summary', 'report']"
kd = f"Поддерживаемые области знаний(knowledgeDomain): {kd_list}"

connector_d = """
Коннектор — программно-аппаратная или алгоритмическая сущность на входе/выходе инструмента, обеспечивающая взаимодействие и оценку качества соединения. Выбирает оптимальное соединение по метрике R(x), максимизируя её значение.
Входной коннектор:
DT_in: тип данных (напр., текст, звук, изображение).
ST_in: семантический тип (напр., температура, описание события).
KD_in: область компетенции (напр., физика, лингвистика).
Выходной коннектор:
DT_out: тип данных (напр., числовой вектор, текст).
ST_out: семантический тип (напр., предсказание, классификация).
KD_out: область знаний (напр., медицина, статистика).

-------------------------------------------------
"""

sem_d = """
Семантический формат данных уточняет смысловое содержание, дополняя их тип. Он включает два уровня: категории семантического содержания и их характеристики.
Категории семантического содержания:
Сырые данные — необработанные.
Спектральный состав — частотное распределение.
Кепстральные коэффициенты — для анализа сигналов.
Речь — устная человеческая речь.
Вопрос — запрос, требующий ответа.
Ответ — решение или информация.
Аннотация — краткое изложение.
Рассуждения — логический анализ.
Отчет — структурированный документ с итогами.
JSON-объект:
semanticCategory: категория содержания ("raw", "specter", "cepstrum", "speak", "question", "answer", "summary", "reasoning", "report").
Вложенный объект с характеристиками категории.
Ключи — строки, значения — строки, числа или объекты.

--------------------------------
"""

data_d = """
Тип данных определяет формат и структуру информации, обрабатываемой коннектором. Они делятся на два уровня: сложные типы (файлы и структурированные представления) и их подтипы.
Файлы хранятся с расширениями, указывающими формат:
.txt — текст,
.json — JSON,
.pdf — документы,
.jpg — изображения JPEG,
.wav — аудио WAV.
Структурированные представления включают:
Математические объекты: матрицы, векторы, тензоры, сигналы.
Пользовательские данные: текст, изображение, звук.
JSON-объект описывает тип данных:
complexType: "file" или "structured".
subtype: уточняет подтип (напр., "txt", "matrix").
Характеристики подтипа в виде пар "ключ-значение".
--------------------------
"""

In [9]:

text_p1 = "Возьми то сообщение что я тебе продиктовал и переведи в текст"
text_p2 = "Объясни почему трава голубая, сделай это голосом!"

structure1 = """
{
   "input_connector":
    {
        "dataType":
        {
          "complexType": "structured",
          "subtype": "sound"
        },

        "semanticType":
        {
          "semanticCategory": "speak",
        },

        "knowledgeDomain": "general"

    },

    "output_connector":
    {
        "dataType":
        {
          "complexType": "file",
          "subtype": "txt"
        },

        "semanticType":
        {
          "semanticCategory": "speak",
        },

        "knowledgeDomain": "general"
    }
}
"""

structure2 = """
{
    "input_connector": {
        "dataType": {
            "complexType": "structured",
            "subtype": "text"
        },
        "semanticType": {
            "semanticCategory": "question"
        },
        "knowledgeDomain": "biology"
    },
    "output_connector": {
        "dataType": {
            "complexType": "structured",
            "subtype": "sound"
        },
        "semanticType": {
            "semanticCategory": "answer"
        },
        "knowledgeDomain": "biology"
    }
}
"""

structure_out = f"""
{{
    "input_connector": {{
        "dataType": {{
            "complexType": выбери либо "structured", либо "file", предоставил ли пользователь файл, если да то "file", иначе "structured"
            "subtype": если на предыдущем этапе был выбран "structured", выбери из этого списка: {s_i}, а если file, из этого {f_i}
        }},
        "semanticType": {{
            "semanticCategory": Выбери одну из этого списка {st_i} исходные данные raw, текстовые и голосовые вопросы question
        }},
        "knowledgeDomain": выбери из этого списка {kd_list}, не используй другие области
    }},
    "output_connector": {{
        "dataType": {{
            "complexType": выбери либо "structured", либо "file", нужно ли в ответ векнуть файл, если да то "file", иначе "structured"
            "subtype": если на предыдущем этапе был выбран "structured", выбери из этого списка: {s_i}, а если file, из этого {f_i}
        }},
        "semanticType": {{
            "semanticCategory": Выбери одну из этого списка {st_o} исходные данные raw, текстовые и голосовые ответы answer
        }},
        "knowledgeDomain": выбери из этого списка {kd_list}, не используй другие области
    }}
}}
"""


p = "Преобразуй запрос в json. Структура должна определять входной и выходной коннекторы, а также области знаний коннекторов."
p += "Выведи только json, без лишней информации. Учти что поле complexType может принимать только 2 значения file и structured"
p += f"Описание коннектора: {connector_d}, {sem_d}, {data_d} Если доменая область не предполагает знания какой-то узкой тематики, то ставь general(для общих знаний)"
p += "Подсказки: голос это звук, отчет - это файл (docx (редактируемый, можно вносить изменения) и pptx (с картинками, слайдами) или pdf(статичный))"
p += f"\nЗапрос 1: {text_p1}\n Правильная структура: {structure1}\n\n\n"
p += f"\nЗапрос 2: {text_p2}\n Правильная структура: {structure2}\n\n"
p += "Формат ответа следующий: " + structure_out

In [11]:
qa_pipeline = pipeline(
    "text-generation",
    model=model, #model,#inference_model,
    tokenizer=tokenizer,#inference_tokenizer,
    device_map="cuda",
    max_length=512,
    do_sample=True,
    top_k=50,
    top_p=0.9
)

Device set to use cuda:0


In [12]:
def translate_task(task_text):
    prompt = qa_pipeline.tokenizer.apply_chat_template([{"role": "user", "content": f"{p}\n\nЗапрос: {task_text}"}],
                                                       tokenize=False, add_generation_prompt=True)
    generated = qa_pipeline(prompt, max_length=2500, num_return_sequences=1)[0]["generated_text"]
    return generated.split("Запрос:")[1].split("<|im_start|>assistant")[1]

In [13]:
new_question = "Дан звуковой сигнал, содержащий речь. Визуализируй его спектрограмму, выделив наиболее заметные участки по интенсивности и частоте. Полученное изображение должно отражать изменение спектра во времени."
print(translate_task(new_question))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
`generation_config` default values have been modified to match model-specific defaults: {'top_k': 20}. If this is not desired, please set these values explicitly.



{    
   "input_connector": 
    {
        "dataType":
        {
          "complexType": "structured",
          "subtype": "sound"
        },
        
        "semanticType":
        {
          "semanticCategory": "cepstrum",
        },
        
        "knowledgeDomain": "general"
        
    },

    "output_connector": 
    {
        "dataType":
        {
          "complexType": "file",
          "subtype": "png"
        },
        
        "semanticType":
        {
          "semanticCategory": "cepstrum",
        },
        
        "knowledgeDomain": "general"
    },
}





